In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import plotly as pl
from scipy.optimize import minimize
from statsmodels.sandbox.distributions.extras import ACSkewT_gen

In [ ]:
np.random.seed(20121020)

pl.offline.init_notebook_mode(connected=True)

In [ ]:
def objective_function_test(parameters):

    a, b = parameters

    return a + b


minimize(objective_function_test, (8, 8))

In [ ]:
def objective_function(parameters, model, independent_variables):

    degree_of_freedom, shape, location, scale = parameters

    return -model.pdf(
        independent_variables, 2 ** degree_of_freedom, shape, loc=location, scale=scale
    ).sum()

In [ ]:
location = 0

scale = 1

random_variables = np.random.normal(loc=location, scale=scale, size=100)

over = (random_variables.max() - random_variables.min()) * 0

independent_variables = np.linspace(
    random_variables.min() - over, random_variables.max() + over, 32
)

model = ACSkewT_gen()

fit_result = model.fit(random_variables, loc=location, scale=scale)

for method in (
    "Nelder-Mead",
    "Powell",
    "CG",
    "BFGS",
    #     "Newton-CG",
    "L-BFGS-B",
    "TNC",
    #     "COBYLA",
    "SLSQP",
    "trust-constr",
    #     "dogleg",
    #     "trust-ncg",
    #     "trust-exact",
    #     "trust-krylov",
):

    print(method)

    minimize_result = minimize(
        objective_function,
        (1, 0, location, scale),
        args=(model, independent_variables),
        method=method,
    )

    pl.offline.iplot(
        {
            "layout": {},
            "data": [
                {
                    "type": "histogram",
                    "name": "Random Variable Distribution",
                    "histnorm": "probability density",
                    "x": random_variables,
                },
                {
                    "type": "scatter",
                    "name": "Fit (fit)",
                    "x": independent_variables,
                    "y": model.pdf(
                        independent_variables,
                        fit_result[0],
                        fit_result[1],
                        loc=fit_result[2],
                        scale=fit_result[3],
                    ),
                },
                {
                    "type": "scatter",
                    "name": "Fit (minimize)",
                    "x": independent_variables,
                    "y": model.pdf(
                        independent_variables,
                        2 ** minimize_result.x[0],
                        minimize_result.x[1],
                        loc=minimize_result.x[2],
                        scale=minimize_result.x[3],
                    ),
                },
            ],
        }
    )